In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from scipy.io import loadmat
from scipy.optimize import minimize

In [2]:
def one_hot(Y):
    y = Y.reshape(-1)
    res = []
    for i in y:
        temp = np.zeros(10)
        temp[i-1] = 1
        res.append(temp)
    return res

In [3]:
path_1 = 'ex4data1.mat'
path_2 = 'ex4weights.mat'
data = loadmat(path_1)
theta = loadmat(path_2)
X = data['X']
Y = data['y']
theta_1 = theta['Theta1']
theta_2 = theta['Theta2']
print(data.keys())
print(theta.keys())
y_one_hot = one_hot(Y)
y_one_hot = np.array(y_one_hot)
X = np.insert(X,0,np.ones(X.shape[0]),axis=1)

dict_keys(['__header__', '__version__', '__globals__', 'X', 'y'])
dict_keys(['__header__', '__version__', '__globals__', 'Theta1', 'Theta2'])


In [4]:
def ser(theta_1,theta_2):
    return np.concatenate((np.ravel(theta_1),np.ravel(theta_2)))

In [5]:
def diser(theta_all):
    return theta_all[:25*401].reshape(25,401) , theta_all[25*401:].reshape(10,26)

In [6]:
def sigmoid(X):
    return 1 / (1 + np.exp(-X))

In [7]:
def feed_forward(theta_all,X):
    theta_1,theta_2 = diser(theta_all)
    a1 = X
    z2 = a1@theta_1.T
    a2 = sigmoid(z2)
    a2 = np.insert(a2,0,np.ones(a2.shape[0]),axis=1)

    z3 = a2@theta_2.T
    a3 = sigmoid(z3)

    return a1,z2,a2,z3,a3

In [8]:
def cost(theta, X, y):
    h = feed_forward(theta, X)[-1]
    tmp = -y * np.log(h) - (1-y) * np.log(1-h)
    return tmp.sum() / y.shape[0]

In [9]:
def reg_cos(theta_all,X,Y,lamda):
    theta_1 ,theta_2 = diser(theta_all)
    cos = cost(theta_all,X,Y)
    reg_1 = np.sum(np.power(theta_1[:,1:],2)) * lamda / ( 2  *  len(X))
    reg_2 = np.sum(np.power(theta_2[:,1:],2)) * lamda / ( 2  *  len(X))
    
    return cos + reg_1 + reg_2  

In [10]:
def sigmoid_gradient(X):
    return sigmoid(X)*(1 - sigmoid(X))

In [11]:
def gradient(theta_all,X,Y):
    theta_1 , theta_2 = diser(theta_all)
    a1,z2,a2,z3,a3 = feed_forward(theta_all,X)
    
    d3 = a3 - Y
    d2 = d3@theta_2[:,1:]*sigmoid_gradient(z2)
    
    D1 = d2.T@a1/len(X)
    D2 = d3.T@a2/len(X)
    return ser(D1,D2)

In [12]:
def reg_gradient(theta_all,X,Y,lamda):
    gr = gradient(theta_all,X,Y)
    D1,D2 = diser(gr)
    theta_1 ,theta_2 = diser(theta_all)
    D1[:,1:] = D1[:,1:] + lamda / len(X) * theta_1[:,1:]
    D2[:,1:] = D2[:,1:] + lamda / len(X) * theta_2[:,1:]
    return ser(D1,D2)

In [13]:
def training(X,Y,lamda):
    theta = np.random.uniform(-0.12,0.12,10285)
    result = minimize(fun=reg_cos,x0=theta,args=(X,Y,1),method='TNC',jac=reg_gradient)
    return result

In [14]:
res = training(X,y_one_hot,1)
res

 message: Converged (|f_n-f_(n-1)| ~= 0)
 success: True
  status: 1
     fun: 0.2971806711828541
       x: [-9.332e-01  2.211e-11 ... -2.129e-01 -5.192e-01]
     nit: 239
     jac: [-8.582e-10  4.421e-15 ... -3.477e-09  3.754e-08]
    nfev: 6326

In [15]:
print(Y.reshape(-1))

[10 10 10 ...  9  9  9]


In [16]:
out = feed_forward(res.x,X)[-1]
y_p = np.argmax(out,axis=1) + 1
print(classification_report(Y.reshape(-1),y_p))
print(y_p)

              precision    recall  f1-score   support

           1       0.99      1.00      1.00       500
           2       1.00      1.00      1.00       500
           3       1.00      0.99      0.99       500
           4       1.00      0.99      1.00       500
           5       1.00      1.00      1.00       500
           6       1.00      1.00      1.00       500
           7       0.99      1.00      1.00       500
           8       1.00      1.00      1.00       500
           9       0.99      0.99      0.99       500
          10       0.99      1.00      1.00       500

    accuracy                           1.00      5000
   macro avg       1.00      1.00      1.00      5000
weighted avg       1.00      1.00      1.00      5000

[10 10 10 ...  9  9  9]


In [17]:
def expand_array(arr):
    return np.array(np.matrix(np.ones(arr.shape[0])).T@np.matrix(arr))

In [24]:
def gradient_check(theta,epsilon,X,Y):
    m = len(theta)
    theta_change = expand_array(theta)
    epsilon_matrix = np.identity(m) * epsilon
    plus_matrix = theta_change + epsilon_matrix
    mins_matrix = theta_change - epsilon_matrix
    approx_grad = []
    for i in range(m):
        approx_grad.append((reg_cos(plus_matrix[i],X,Y,1) - reg_cos(mins_matrix[i],X,Y,1)) / (2 * epsilon))
    approx_grad = np.array(approx_grad)
    analize_grad = reg_gradient(theta,X,Y,1)

    
    diff = np.linalg.norm(approx_grad - analize_grad) / np.linalg.norm(approx_grad + analize_grad)
    print(diff)

In [25]:
the = ser(theta_1,theta_2)
gradient_check(the,0.001,X,y_one_hot)

3.028137994555929e-07
